In [1]:
import pandas as pd
import os

#### Read benchmark data

In [2]:
benchmarks_py = pd.read_csv('python/benchmarks_py.csv')
benchmarks_jl = pd.read_csv('julia/benchmarks_jl.csv')


#### Show Benchmarks

In [3]:
benchmarks_py['Python'] = [round(t) for t in benchmarks_py['Python']]
benchmarks_jl['Julia'] = [round(t) for t in benchmarks_jl['Julia']]

benchmarks = pd.merge(
    benchmarks_py.groupby(by = ['Problem', 'Date']).min(),
    benchmarks_jl.groupby(by = ['Problem', 'Date']).min(),
    how = 'outer', 
    on = 'Problem'
)

benchmarks # in nanoseconds (ns)

,Python,Julia
Problem,,
1,1.461300e+04,7.950000e+02
2,6.884000e+03,1.950000e+02
3,1.547143e+06,1.833000e+03
4,1.433964e+08,2.944538e+07
5,6.716487e+09,3.230000e+02
6,3.843000e+03,2.000000e+00
7,2.329697e+07,2.772021e+06
8,9.265020e+05,2.599800e+04
9,8.510237e+09,8.692800e+04


#### How much faster is Julia compared to Python?

In [4]:
benchmarks.dropna(inplace=True)
benchmarks['Python'] = [round(t) for t in benchmarks['Python']]
benchmarks['Julia'] = [round(t) for t in benchmarks['Julia']]
benchmarks['Faster (x times)'] = [round(p/j, 1) for (p, j) in zip(benchmarks.Python, benchmarks.Julia)]
benchmarks.loc[benchmarks['Faster (x times)'] > 1, ['Faster (x times)']]

,Faster (x times)
Problem,
1,18.4
2,35.3
3,844.0
4,4.9
5,20794078.4
6,1921.5
7,8.4
8,35.6
9,97899.8


#### How much faster is Python compared to Julia?

In [5]:
benchmarks.dropna(inplace=True)
benchmarks['Python'] = [round(t) for t in benchmarks['Python']]
benchmarks['Julia'] = [round(t) for t in benchmarks['Julia']]
benchmarks['Faster (x times)'] = [round(j/p, 1) for (p, j) in zip(benchmarks.Python, benchmarks.Julia)]
benchmarks.loc[benchmarks['Faster (x times)'] > 1, ['Faster (x times)']]

,Faster (x times)
Problem,
22,1.7
29,2.8
